# 의사결정나무 (Decision Tree)

**학습 목표:**
- 스무고개와 같이 직관적인 원리로 동작하는 의사결정나무 모델을 학습합니다.
- 불순도(Impurity) 개념(지니 계수)을 이해하고, 모델이 어떻게 최적의 질문(분기 기준)을 찾는지 알아봅니다.
- 학습된 모델을 **시각화**하여 의사결정 과정을 명확하게 파악합니다.
- 과적합을 방지하기 위한 **가지치기(Pruning)**의 필요성과 방법을 학습합니다. (`max_depth`)
- 모델이 어떤 특성을 중요하게 생각하는지 **특성 중요도(Feature Importance)**를 확인합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score

### (1) 데이터 준비 및 탐색
붓꽃(Iris) 데이터셋을 사용합니다. 꽃받침(sepal)과 꽃잎(petal)의 길이/너비 특징을 사용하여 3가지 붓꽃 품종(Setosa, Versicolor, Virginica)을 분류합니다.

In [ ]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='species')
y_labels = y.map({0: iris.target_names[0], 1: iris.target_names[1], 2: iris.target_names[2]})

# Pairplot으로 특성 간 관계 탐색
sns.pairplot(pd.concat([X, y_labels], axis=1), hue='species', palette='viridis')
plt.show()

### (2) 의사결정나무 모델 학습 및 시각화
별다른 제약을 주지 않고 모델을 학습시켜 최대 성능과 그 구조를 확인합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 의사결정나무 모델 생성 및 학습
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

y_pred = dt_clf.predict(X_test)
print(f"Full Tree Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# 학습된 트리 시각화
plt.figure(figsize=(25, 15))
plot_tree(dt_clf, 
          feature_names=iris.feature_names, 
          class_names=iris.target_names, 
          filled=True, 
          rounded=True,
          fontsize=10)
plt.title("Full Decision Tree Structure", fontsize=20)
plt.show()

### (3) 과적합 제어를 위한 가지치기 (Pruning)
모델이 너무 복잡해져 훈련 데이터에만 과적합되는 것을 막기 위해 가지치기를 수행합니다. `max_depth`는 트리의 최대 깊이를 제한하는 가장 간단하고 효과적인 방법입니다.

In [ ]:
# 트리의 최대 깊이를 3으로 제한
dt_pruned_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_pruned_clf.fit(X_train, y_train)

y_pred_pruned = dt_pruned_clf.predict(X_test)
print(f"Pruned Tree (max_depth=3) Accuracy: {accuracy_score(y_test, y_pred_pruned):.4f}")

# 가지치기된 트리 시각화
plt.figure(figsize=(15, 10))
plot_tree(dt_pruned_clf, 
          feature_names=iris.feature_names, 
          class_names=iris.target_names, 
          filled=True, 
          rounded=True)
plt.title("Pruned Decision Tree (max_depth=3)", fontsize=16)
plt.show()
print("모델이 더 단순해졌지만 성능은 거의 동일하게 유지되어, 더 일반화된 모델이라고 볼 수 있습니다.")

### (4) 특성 중요도 (Feature Importance)
모델이 분류 작업을 수행할 때 어떤 특성을 더 중요하게 사용했는지 확인할 수 있습니다. 특성 중요도는 모든 노드에서 해당 특성을 사용하여 불순도를 감소시킨 양의 총합으로 계산됩니다.

In [ ]:
importances = dt_clf.feature_importances_
importance_df = pd.DataFrame({'Feature': iris.feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('Feature Importances from Decision Tree')
plt.show()
print("petal length와 petal width가 품종을 분류하는 데 가장 중요한 특성임을 알 수 있습니다.")